## All the import

In [67]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
from keras.layers import Merge

## Load the data

In [68]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

In [69]:
def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1  

    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice3[:nb_train])

    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice3[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing

## Calculate the performance

In [70]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

## Create The Model

In [71]:
def get_cnn_network(matrixsize = 6, nbfilter = 24, matrixsize2 = 7, nbfilter2 = 6):    
    k = matrixsize
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    W[:,:,0,1] = I2
    W[:,:,0,2] = M
    W[:,:,0,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2
    print('configure cnn network')
 
    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))    
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    
    k2 = matrixsize2
    I = np.eye(k2)
    M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
    I2=np.zeros((k2,k2))
    M2=np.zeros((k2,k2))
    for j in range(k2):
        I2[:,j] = I[:,k2-j-1]
        M2[:,j] = M[:,k2-j-1]   
    
    Z = np.zeros((k2,k2,nbfilter,nbfilter2))
    
    for u in range(nbfilter):
        Z[:,:,u,0] = I
        Z[:,:,u,1] = I2
        Z[:,:,u,2] = M
        Z[:,:,u,3] = M2
        for p in range(4,nbfilter2):
            Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

    model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter2)]))
    print(model.output_shape)    
    model.add(Flatten())
        
    return model

In [72]:
def get_cnn_network_seq(nbfilter = 64, kernelsize = 7):        
    model = Sequential()
    model.add(Conv1D(filters = nbfilter, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None)))
    model.add(MaxPooling1D(pool_size=6))
    print(model.output_shape)
    model.add(Flatten())
    return model

In [73]:
def create_model(matrixsize11 = 6, nbfilter11 = 24, matrixsize12 = 7, nbfilter12 = 6, 
                 matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7, nbfilter22 = 6,
                 nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                 Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    training_net=[]
    training_net1 = []
    training_net2 = []
    training_net1.append(get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12))
    training_net1.append(get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22))
    training_net2.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net2.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    
    
    model2 = Sequential()
    model2.add(Merge(training_net2, mode ='concat'))
    model2.add(Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model2.add(BatchNormalization())
    model2.add(Activation('relu'))
    #model2.add(Dropout(0.1))
    
    model1 = Sequential()
    model1.add(Merge(training_net1, mode ='concat'))
    model1.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    #model1.add(Dropout(0.1))
    model1.add(BatchNormalization())
    model1.add(Activation('relu'))
    
    training_net.append(model1)
    training_net.append(model2)
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
    
    model.add(Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
          
    return model

## Train and Evaluate the model

In [74]:
Data = load_data(165000,6000,5000)

26023
24229
37991
26023
24229
37993


MemoryError: 

In [ ]:
def train_model(training = [], y = [], batch_size=16, epochs=6, verbose1 = 1, verbose2 = 1,
                              validation = [], val_y = [], testing = [], matrixsize11 = 6, nbfilter11 = 24, 
                              matrixsize12 = 7, nbfilter12 = 6, matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7,
                              nbfilter22 = 6, nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                              Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    model = create_model(matrixsize11, nbfilter11, matrixsize12, nbfilter12, matrixsize21, nbfilter21, 
                         matrixsize22, nbfilter22, nbfilter1, kernel_size1, nbfilters2,kernel_size2,
                         Dense1, Dense2, Dense3, Dense4, Dense5)
    earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose = verbose1)
    print('model training')
    model.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("batch_size : ", batch_size)
    print("epochs : ",epochs)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])
    
    return model


## Save The Model

In [ ]:
def save_model(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    json_file.close()
    print("Saved model to disk")

## Load The Model

In [ ]:
def load_model():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    json_file.close()
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model

## Run

In [ ]:
model = train_model(Data[0],Data[1],16,6,1,1,Data[2],Data[3],Data[4],6,24,7,6,6,24,7,6,64,7,64,7,512,128,512,128,64)
save_model(model)